# Mediation, Moderation, and Conditional Process Analyses

Mediation and moderation are two distinct concepts in statistics that help in understanding the relationships among three or more variables. Mediation deals with explaining the underlying process through which an independent variable (X) influences a dependent variable (Y). Moderation, on the other hand, addresses how the relationship between X and Y changes based on the level of a third variable, known as the moderator (W).

**Mediation Analysis**:

In mediation analysis, we are interested in whether the relationship between X and Y is explained (or mediated) through another variable, called the mediator (M).

- **Path a**: The independent variable (X) affects the mediator (M). This is the "a" path.
- **Path b**: The mediator (M) affects the dependent variable (Y). This is the "b" path.
- **Path c'**: There might also be a direct effect of X on Y that doesn't go through the mediator. This is the "c'" path or direct effect.

```
X ---(a)---> M ---(b)---> Y
 \                        /
  ---------(c')----------
```

The indirect effect is a*b, which is the portion of the effect of X on Y that is transmitted through the mediator M. The total effect of X on Y is the sum of the indirect effect and the direct effect (c').

**Moderated Mediation Analysis**:

Moderated mediation incorporates the idea that the mediation process itself can be contingent upon the levels of another variable (W), known as the moderator. In other words, the indirect effect of X on Y through M may change depending on the level of W.

```
X ---(a)---> M ---(b)---> Y
|            | 
|------(W)---|
 \                        /
  ---------(c')----------
```

**Interpreting the Results**:

In the results of a moderated mediation analysis, you'll encounter several terms:

- ACME (control/treated): Average Causal Mediation Effects. This represents the average effect of the independent variable on the dependent variable through the mediator.
- ADE (control/treated): Average Direct Effects. This represents the average direct effect of the independent variable on the dependent variable not through the mediator.
- Total effect: Sum of ACME and ADE, representing the total effect of the independent variable on the dependent variable.
- Prop. mediated (control/treated): Proportion of the total effect that is mediated by the mediator.
- The 'control' and 'treated' distinction in ACME and ADE accounts for possible treatment or intervention that could affect the mediation. If there's no treatment/intervention involved, they will be the same.
- Lower CI bound and Upper CI bound represent the lower and upper bounds of the confidence interval for the estimates.
- P-value: Indicates the statistical significance of the estimates. A lower p-value (<0.05) typically indicates a statistically significant effect.

- Control: baseline regression without effect of the mediator
- Treatment: mediated regression wherein the mediator is accounted for
 

**Identifying Partial Mediations**:

In partial mediation, the mediator explains some, but not all, of the relationship between the independent variable (exposure) and the dependent variable (outcome). Here's how you can identify partial mediation:

Significant Direct Effect (ADE): In partial mediation, the direct effect of the independent variable on the dependent variable remains significant even after accounting for the mediator. This means that there is still a direct path from the independent variable to the dependent variable that is not explained by the mediator.

Significant Indirect Effect (ACME): The indirect effect through the mediator must also be significant. This means that the mediator is explaining part of the relationship between the independent variable and the dependent variable.

Total Effect > Direct Effect: The total effect of the independent variable on the dependent variable (before the mediator is added to the model) is generally greater than the direct effect (after accounting for the mediator). This shows that the mediator is explaining some portion of the effect.

In the results you usually get from mediation analysis, check for the following:

ADE (average direct effect) should be significant.
ACME (average causal mediation effect) should be significant.
The total effect should be greater than the direct effect, indicating that some portion of the effect is being accounted for by the mediator.
In summary, partial mediation is present when both the direct and indirect paths are significant, but the inclusion of the mediator in the model reduces the direct effect of the independent variable on the dependent variable (without rendering it non-significant).

**Identifying a Mediated Exposure**:

If the exposure was being mediated, you would expect to see:

A significant ACME (Average Causal Mediation Effect) - This indicates that the mediator is having a significant effect in the relationship between the exposure and the outcome.

A significant proportion mediated - This tells you the proportion of the total effect that is mediated. If this is significant, it implies that a noteworthy part of the relationship between the exposure and the outcome is occurring through the mediator.

Also, keep in mind that the proportion mediated can be positive or negative, and this would indicate whether the mediator is increasing or decreasing the effect of the exposure on the outcome.

In [ ]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

In [ ]:
analysis = 'mediated_moderation_analysis'
data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_analyses/ses-01/sub-all/all_data/all_metadata_spreadsheet.csv'
out_dir = os.path.join(data_path.split('.')[0], f'{analysis}')

save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

In [ ]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
data_df = preprocess_colnames_for_regression(pd.read_csv(data_path))
data_df.head(1)

Data Cleaning

In [ ]:
#Select specific subgroup
# outlier_index = (data_df['percent_change_adascog11'] <= -50)
# data_df = data_df.loc[outlier_index, :]

#Remove outlier
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

data_df.head(2)

In [ ]:

#Rename the outcome variable
outcome_variable = data_df.pop('%_Change_from_baseline_(ADAS_Cog11)')
data_df['outcome'] = outcome_variable

In [ ]:
#Standardize the data
preserved_df = data_df.copy()
from sklearn.preprocessing import StandardScaler


# Remove anything you don't want to standardize
cols_not_to_standardize = ['Patient_#_CDR,_ADAS', 'Disease']
# Select the columns to be standardized
if cols_not_to_standardize[0] is not None:
    cols_to_standardize = [col for col in data_df.columns if col not in cols_not_to_standardize]
else:
    print('Will not standardize')
# Standardize
scaler = StandardScaler()
data_df[cols_to_standardize] = scaler.fit_transform(data_df[cols_to_standardize])




#Drop NAN
# data_df = data_df.dropna()
data_df

In [ ]:
# One-hot encode as needed
# data_df['interaction'] = data_df['Subiculum_Grey_Matter']*data_df['Subiculum_Connectivity']
data_df['Disease'] = np.where(data_df['Disease'] == 'Alzheimer', 1, 0)
# data_df['Age'] = np.where(data_df['Age'] <= 65, 0, 1)

data_df

Select Specific Rows

In [ ]:
#Drop a specific set of rows
# data_df = data_df[data_df['Age'] > 65]
data_df = data_df[data_df['Disease'] == 1]
data_df

Select Specific Columns

In [ ]:
print(data_df.columns)


In [ ]:
# Tailor the Dataframe
# data_df = data_df.loc[:, ['Age', 'Mesial_Temporal_Grade', 'Frontal', 'Temporal',
#        'Parietal', 'Occipital', 'Cerebellar', 'Subiculum_Connectivity',
#        'Hippocampal_CSF', 'Hippocampal_Grey_Matter',
#        'Hippocampal_White_Matter', 'outcome']]
data_df

Single Variable Mediation Analysis

Model Shape:
```
 Age ---(an)---> Atrophy Region n ---(bn)---> Outcome
  \                                          /
   -------------------(c')-------------------
```

Description
- This uses partial regressions to essentially (explain in a reductive manner) to what degree an independent variable's relationship with a dependent variable can be replaced by a 'mediator variable'. 
- Said another way, how much is the effect of x upon y mediated by z?

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

def perform_mediation_analysis(dataframe, exposure, mediator, dependent_variable):
    # Step 1: Fit the mediator model
    # Mediator ~ Independent Variable
    mediator_model = GLM.from_formula(f"{mediator} ~ {exposure}", data=dataframe, family=Gaussian())
    
    # Step 2: Fit the outcome model
    # Dependent Variable ~ Independent Variable + Mediator
    outcome_model = GLM.from_formula(f"{dependent_variable} ~ {exposure} + {mediator}", data=dataframe, family=Gaussian())
    
    # Step 3: Perform mediation analysis
    med = Mediation(outcome_model, mediator_model, exposure=exposure, mediator=mediator)
    try:
        med_result = med.fit()
        # Step 4: Print the results
        print(f'Mediation of {exposure} by {mediator}:')
        print(med_result.summary())
    except:
        print(f"Mediation failed, perfect separation detected. Aborting assessment of {mediator}.")

# Example usage:
# Assuming your DataFrame is named df, and the columns of interest are 'age', 'brain_atrophy', and 'dependent_variable'
# perform_mediation_analysis(data_df, 
                        #    exposure = 'age', 
                        #    mediator = 'brain_atrophy', 
                        #    dependent_variable='dependent_variable')


In [ ]:
data_df.columns

In [ ]:
perform_mediation_analysis(data_df, 
                           exposure = 'Age', 
                           mediator = 'Hippocampus', 
                           dependent_variable='outcome')

Run the above code on an entire spreadsheet

In [ ]:
[perform_mediation_analysis(data_df, exposure='Age', mediator=f'{col}', dependent_variable='outcome') for col in data_df.columns]

__________
**Multivariate Mediation Analysis (Independent)**

**NOTE**
This is not a proper 'multivariate mediation analysis'.
It does something slightly different: it performs individual mediation analyses for each mediator, while controlling for the exposure. This is different from assessing the conglomerate mediation effect of all mediators acting in unison.


To investigate whether the effect of an independent variable on an outcome variable is fully mediated through multiple variables (e.g., atrophy in different brain regions), you would want to conduct a multiple mediator analysis.

In a multiple mediator analysis, you examine the indirect effects of an independent variable (in this case, age) on a dependent variable through several mediators simultaneously. This allows you to estimate the portion of the effect of the independent variable that is transmitted through each mediator and whether, when taken together, these mediators account for the entirety of the effect.

In your case, the different atrophy regions will serve as multiple mediators. Here’s how this might look diagrammatically:

```
 Age ---(a1)---> Atrophy Region 1 ---(b1)---> Outcome
  |           |
 Age ---(a2)---> Atrophy Region 2 ---(b2)---> Outcome
  |           |
   ...       ...
 Age ---(an)---> Atrophy Region n ---(bn)---> Outcome
  \                                          /
   -------------------(c')-------------------
```

In this example, each atrophy region (1 through n) is a mediator. The indirect effect of age on the outcome through each atrophy region is a_i * b_i, and the total indirect effect is the sum of all these individual indirect effects. If this total indirect effect is significant and accounts for most of the effect of age on the outcome, and the direct effect (c') is non-significant, it indicates full mediation.

You can perform multiple mediator analysis using the same statistical tools you use for single mediator analysis, but you will need to include multiple mediators in your model. This can be done using statistical software such as R, SAS, or Python's statsmodels. I have implemented this below using statsmodels.

Here's how the mediation model is constructed in conceptual terms:

1. Fit a model for each mediator (Atrophy Region 1, Atrophy Region 2, ..., Atrophy Region n) as a function of the independent variable, Age.
2. Fit a model for the outcome as a function of Age and all atrophy regions.
3. Estimate the indirect effect of Age on the outcome through each atrophy region and sum them to get the total indirect effect.
4. Compare the total indirect effect to the total effect of Age on the outcome to determine the proportion mediated.

By performing this analysis, you can ascertain whether the combined atrophy in different regions fully mediates the effect of age on the outcome variable. If you also have a moderation effect to consider (such as subiculum connectivity), this would be a moderated multiple mediation analysis. In this case, the indirect effects and the direct effect may change at different levels of the moderator.

I have generated code which will run mutliple mediation analyses independently, but will therefor not be able to tell you how they sum to contribute to the proportion of the independent variable mediated by them. this is perform_individual_multiple_mediator_analysis. Python does not have a combined multiple mediator analysis supported, so I have developed one. It is perform_combined_multiple_mediator_analysis. Please use with caution. 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

def perform_individual_multiple_mediator_analysis(dataframe, exposure, mediator_list, outcome):
    # Check if mediator_cols is a list, if not, convert it to a list
    if not isinstance(mediator_list, list):
        mediator_list = [mediator_list]
    
    # Convert mediator columns into formula expression
    mediator_expression = ' + '.join(mediator_list)
    
    # Step 1: Fit the mediator models
    # Mediators ~ Independent Variable
    mediator_models = [GLM.from_formula(f"{mediator} ~ {exposure}", data=dataframe, family=Gaussian()) for mediator in mediator_list]
    
    # Step 2: Fit the outcome model
    # Dependent Variable ~ Independent Variable + Mediators
    outcome_model = GLM.from_formula(f"{outcome} ~ {exposure} + {mediator_expression}", data=dataframe, family=Gaussian())
    
    # Step 3: Perform mediation analysis for each mediator
    for mediator, mediator_model in zip(mediator_list, mediator_models):
        med = Mediation(outcome_model, mediator_model, exposure=exposure, mediator=mediator)
        med_result = med.fit()
        
        # Step 4: Print the results for each mediator
        print(f"Mediation analysis for mediator: {mediator}")
        print(med_result.summary())
        print("\n")

In [ ]:
data_df.columns

In [ ]:
perform_individual_multiple_mediator_analysis(data_df, 
                                   exposure='Age', 
                                   mediator_list=['Hippocampus', 'Cerebellum', 'Frontal', 'Parietal',
       'Temporal', 'Occipital'], 
                                   outcome='outcome')

_____
**Multivariate Mediation Analysis**
Based on Preacher and Hayes 2008, Asymptotic Asymptotic and resampling strategies for assessing and comparing indirect effects in multiple mediator models

**NOTE**: This is a proper multivariate mediation analysis

Model Explanation:
In a multiple mediation analysis, you are interested in investigating the indirect effects of an independent variable (exposure) on a dependent variable (outcome) through more than one mediator simultaneously. You are also interested in the direct effect of the independent variable on the dependent variable, controlling for the mediators. This analysis helps in understanding how different mediators may carry the influence of an independent variable to a dependent variable.

Model Form:
```
Independent Variable (IV) -----(c')------> Dependent Variable (DV)
    |        |        |                       ^      ^         ^
    |        |        | (a1)                  |      |         |
    |        |        v                       |      |         |
    |        |        Mediator 1 --(b1)-------|      |         |
    |        |                                       |         |
    |        | (a2)                                  |         |
    |        v                                       |         |
    |    Mediator 2 --------(b2)---------------------|         |
    |                                                          |
    ...                                                        |
    |                                                          |
    | (ak)                                                     |
    v                                                          |
Mediator k --------(bk)----------------------------------------|
```
In this diagram:

IV represents the independent variable.
Mediator 1, Mediator 2, ..., Mediator k represent the k mediators in the model.
DV represents the dependent variable.
The path labeled a1 represents the effect of the IV on Mediator 1. Similarly, a2 represents the effect of the IV on Mediator 2, and so on.
The path labeled b1 represents the effect of Mediator 1 on the DV, controlling for other mediators and the IV. Similarly, b2 represents the effect of Mediator 2 on the DV, controlling for other mediators and the IV, and so on.
The path labeled c' represents the direct effect of the IV on the DV, controlling for all the mediators.

**Technical Notes**

This is not formally implemented in Python as well as in R, so this is Calvin's custom code. Use with care.
This is a more 'true' multivariate mediation analysis than the one above

**Technical Explanation**
The perform_multiple_mediation_analysis function is designed to perform a multiple mediation analysis to estimate the joint indirect effects of an exposure variable through multiple mediators on a dependent variable. This is achieved through the use of bootstrapping.

Bootstrapping is a statistical technique where resampling of the dataset is done with replacement to estimate the sampling distribution of a statistic. In the case of multiple mediation analysis, it's used to estimate the joint indirect effects of mediators.

In the function, for each bootstrap sample, the dataset is resampled with replacement. For each mediator, a simple linear regression is performed to estimate the path from the exposure variable to the mediator, denoted as 'a' paths. The product of these 'a' paths is computed, as this represents the joint effect of the exposure on all the mediators.

Next, a multiple regression is performed with the dependent variable as the response, and the mediators and exposure as predictors. The coefficients for the mediators represent the effect of the mediators on the dependent variable, controlling for the exposure, denoted as 'b' paths. The sum of these 'b' paths is computed.

The indirect effect for the bootstrap sample is then calculated as the product of the 'a' paths and the 'b' path sum. This process is repeated for a specified number of bootstrap samples (default is 5000), and the mean indirect effect is calculated from these bootstrap samples.

Finally, the function prints the mean indirect effect and its 95% confidence interval, which is estimated using the 2.5th and 97.5th percentiles of the bootstrap indirect effects.

This approach provides an approximate estimate of the joint indirect effect through multiple mediators and allows for the calculation of confidence intervals around this effect.

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, total_indirect_effects, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """

    ab_path_values = np.array(ab_paths)
    total_indirect_effects = np.array(total_indirect_effects)

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths[i]) * ab_path_values[:, i] <= 0) for i in range(len(mean_ab_paths))]


    # Calculate mean indirect effect and confidence intervals for the total indirect effect
    mean_total_indirect_effect = np.mean(total_indirect_effects)
    lower_bound_total = np.percentile(total_indirect_effects, 2.5)
    upper_bound_total = np.percentile(total_indirect_effects, 97.5)
    p_value_total = np.mean(total_indirect_effects > 0) if np.mean(total_indirect_effects) < 0 else np.mean(total_indirect_effects <= 1)

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': np.concatenate((mean_ab_paths, [mean_total_indirect_effect])),
        '2.5th Percentile': np.concatenate((lower_bounds, [lower_bound_total])),
        '97.5th Percentile': np.concatenate((upper_bounds, [upper_bound_total])),
        'P-value': ab_path_p_values + [p_value_total]
    }, index=mediators + ['Total Indirect Effect'])

    return result_df


def perform_multiple_mediation_analysis(dataframe, exposure, mediators, dependent_variable, bootstrap_samples=5000):
    """
    Performs a multiple mediation analysis by estimating the joint indirect effects of an exposure variable
    through multiple mediators on a dependent variable using bootstrapping.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable.
    - mediators: list, column names of the mediator variables.
    - dependent_variable: str, column name of the dependent variable.
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.

    Example Usage:
    result_df = perform_multiple_mediation_analysis(data_df, exposure='Age',
                                                    mediators=['Brain_Lobe1', 'Brain_Lobe2'],
                                                    dependent_variable='outcome')
    """

    # Perform multiple mediation analysis
    ab_paths, total_indirect_effects = [], []

    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)

        # Paths from exposure to mediators and from mediators to DV
        ab_paths_sample = [ols(f"{mediator} ~ {exposure}", data=sample).fit().params[exposure] *
                           ols(f"{dependent_variable} ~ {mediator} + {exposure}", data=sample).fit().params[mediator]
                           for mediator in mediators]

        # Sum the individual indirect effects for this bootstrap sample
        total_indirect_effect = sum(ab_paths_sample)

        # Append the ab paths and total indirect effect to the lists
        ab_paths.append(ab_paths_sample)
        total_indirect_effects.append(total_indirect_effect)

    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, total_indirect_effects, mediators)

    return result_df



In [ ]:
data_df.columns

In [ ]:
result_df = perform_multiple_mediation_analysis(data_df, 
                                    exposure='Age', 
                                    mediators=['Hippocampus', 'Cerebellum', 'Frontal', 'Parietal',
       'Temporal', 'Occipital']
                                               , 
                                    dependent_variable='outcome',
                                    bootstrap_samples=10000)
result_df

# First Stage Model Moderated Mediation Analyses

Model Structure:
```
 #Age ---(a1)------> Mediator ---(b1)---> Outcome
  |       |                                                                        
   ---(Moderator)
```
This is a 'first-stage model'

First Stage Moderated Mediation: basically just checks if the mediation is dependent on a moderator relating the independent variable to the mediator. Essentially, it's an interaction effect. 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

def first_stage_moderated_mediation(dataframe, exposure, moderator, mediator, dependent_variable):
    """
    This function performs a moderated_mediation analysis on the provided data.
    
    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the independent variable (e.g., 'Age').
    - moderator: str, column name of the moderator variable, the variable interacting with the independent variable (e.g., 'Subiculum_Connectivity').
    - mediator: str, column name of the mediator variable, the variable acting through the independent variable (e.g., 'Hippocampus').
    - dependent_variable: str, column name of the dependent variable (e.g., 'outcome').
       
    Example Usage:
    perform_moderation_analysis(data_df, independent_variable='Age', moderator='Subiculum_Connectivity', mediator='Hippocampus', dependent_variable='outcome')
    """
    
    # Step 1: Fit the mediator model with interaction term
    # Mediator ~ IV + Moderator + IV*Moderator
    mediator_model = GLM.from_formula(f"{mediator} ~ {exposure} + {moderator} + {exposure}:{moderator}", data=dataframe, family=Gaussian())
    
    # Step 2: Fit the outcome model with mediator
    # Dependent Variable ~ IV + Moderator + IV*Moderator + Mediator
    outcome_model = GLM.from_formula(f"{dependent_variable} ~ {exposure} + {moderator} + {exposure}:{moderator} + {mediator}", data=dataframe, family=Gaussian())
    
    # Step 3: Perform mediation analysis
    med = Mediation(outcome_model, mediator_model, exposure=exposure, mediator=mediator)
    try:
        med_result = med.fit()
        print(f'Exposure {exposure} moderated by {moderator} mediated by {mediator}')
        print(med_result.summary())
        return med_result
    except:
        print(f'Perfect separation detected, aborting {mediator}')
        return


In [ ]:
data_df.columns

In [ ]:
# Example usage:
# Assuming your DataFrame is named data_df, and you're investigating if 'Hippocampus' mediates the relationship between 'Age' and 'outcome', with 'Subiculum_Connectivity' as a moderator.
mediation_results = first_stage_moderated_mediation(data_df, 
                                     exposure='Age', 
                                     moderator='Subiculum_Connectivity', 
                                     mediator='Hippocampus_Damage_Score', 
                                     dependent_variable='outcome')
mediation_results.summary()

In [ ]:
[first_stage_moderated_mediation(data_df, exposure='Age', moderator='Subiculum_Connectivity', mediator=f'{col}', dependent_variable='outcome') for col in data_df.columns]

_____________________
Second Stage Model Moderated Mediation

The moderation is on the path from the independent variable (age) to the mediator (atrophy region). This type of model is often termed as a first-stage moderated mediation. The moderation in this case can show if the relationship between age and the atrophy region varies at different levels of another variable (the moderator).

This is model is not natively supported in Python, thus I have developed it myself. 
Highly complex and experimental model. Use with caution. 


Model Structure:
```
 #Age ---(a1)------> Mediator ---(b1)---> Outcome
  |                                  |
  |                                  |
   |                                 |
   ---(Moderator)---------------------
```
Second Stage Moderated Mediation: When the moderation is occurring in the second part of the mediation process, it is called second stage moderated mediation. In this case, the relationship between the mediator (M) and the dependent variable (DV) depends on the moderator. This is sometimes referred to as "moderated b path."


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

def second_stage_moderated_mediation(dataframe, exposure, moderator, mediator, dependent_variable):
    """
    This function performs a moderated mediation analysis on the provided data.
    
    Parameters:
    - dataframe: DataFrame containing the data.
    - independent_variable: str, column name of the independent variable (e.g., 'Age').
    - moderator: str, column name of the moderator variable, the variable interacting with the independent variable (e.g., 'Subiculum_Connectivity').
    - mediator: str, column name of the mediator variable, the variable acting through the independent variable (e.g., 'Hippocampus').
    - dependent_variable: str, column name of the dependent variable (e.g., 'outcome').
       
    Example Usage:
    perform_moderated_mediation_analysis(data_df, independent_variable='Age', moderator='Subiculum_Connectivity', mediator='Hippocampus', dependent_variable='outcome')
    """
    
    # Step 1: Fit the mediator model with interaction term
    # Mediator ~ IV
    mediator_model = GLM.from_formula(f"{mediator} ~ {exposure}", data=dataframe, family=Gaussian())
    
    # Step 2: Fit the outcome model with mediator
    # Dependent Variable ~ IV + Moderator + IV*Moderator + Mediator
    outcome_model = GLM.from_formula(f"{dependent_variable} ~ {exposure} + {moderator} + {mediator}:{moderator} + {mediator}", data=dataframe, family=Gaussian())

    # Step 3: Perform mediation analysis
    med = Mediation(outcome_model, mediator_model, exposure=exposure, mediator=mediator)
    try:
        med_result = med.fit()
        print(f'Exposure {exposure} mediated by {mediator} and then moderated by {moderator}')
        print(med_result.summary())
        return med_result
    except:
        print(f'Perfect separation detected, aborting {mediator}')
        return np.NaN
     

In [ ]:
med_result = second_stage_moderated_mediation(data_df, 
                                exposure='Age', 
                                moderator='Subiculum_Connectivity', 
                                mediator='Hippocampus_Damage_Score', 
                                dependent_variable='outcome')
med_result.summary()

Run the code above for an entire spreadsheet

In [ ]:
[second_stage_moderated_mediation(data_df, exposure='Age', moderator='Subiculum_Connectivity', mediator=f'{col}', dependent_variable='outcome') for col in data_df.columns]

# Mediated Moderation Analyses

**Classic Mediated Moderation**

The indirect effect of the exposure on the outcome through the mediator is conditional based on the levels of the moderator.

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian
from statsmodels.stats.mediation import Mediation

def perform_mediated_moderation_analysis(dataframe, exposure, mediator, moderator, dependent_variable):
    # Step 1: Fit the mediator model
    # Mediator ~ Independent Variable * Moderator
    mediator_model = GLM.from_formula(f"{mediator} ~ {exposure} * {moderator}", data=dataframe, family=Gaussian())
    
    # Step 2: Fit the outcome model
    # Dependent Variable ~ Independent Variable + Mediator + Independent Variable * Moderator + Mediator * Moderator
    outcome_model = GLM.from_formula(f"{dependent_variable} ~ {exposure} + {mediator} + {exposure}:{moderator} + {mediator}:{moderator}", data=dataframe, family=Gaussian())
    
    # Step 3: Perform mediation analysis
    med = Mediation(outcome_model, mediator_model, exposure=exposure, mediator=mediator)
    try:
        med_result = med.fit()
        # Step 4: Print the results
        print(f'Mediated Moderation of {exposure} by {mediator} with {moderator} as the moderator:')
        print(med_result.summary())
    except:
        print(f"Mediated Moderation failed, perfect separation detected. Aborting assessment of {mediator}.")

# Example usage:
# Assuming your DataFrame is named df, and the columns of interest are 'DBS', 'brain_atrophy', 'age', and 'dependent_variable'
# perform_mediated_moderation_analysis(data_df, 
#                                      exposure = 'DBS', 
#                                      mediator = 'brain_atrophy', 
#                                      moderator = 'age',
#                                      dependent_variable='dependent_variable')


In [ ]:
data_df.columns

Run Everything!

In [33]:
for col in data_df.columns[6:-1]:
    print(perform_mediated_moderation_analysis(data_df, 
                                     exposure='Subiculum_Connectivity', 
                                     mediator=f'{col}', 
                                     moderator='Age', 
                                     dependent_variable='outcome'))

100%|██████████| 5000/5000 [00:30<00:00, 162.58it/s]


5000
                 Point Estimate  2.5th Percentile  97.5th Percentile  P-value
Frontal_Atrophy        0.001085         -0.087468           0.094363    0.498


100%|██████████| 5000/5000 [00:29<00:00, 169.67it/s]


5000
                Point Estimate  2.5th Percentile  97.5th Percentile  P-value
Insula_Atrophy       -0.022052         -0.145237           0.064394   0.3368


100%|██████████| 5000/5000 [00:27<00:00, 181.92it/s]


...

5000
                           Point Estimate  2.5th Percentile  \
Correlated_Memory_Atrophy        -0.01059         -0.154981   

                           97.5th Percentile  P-value  
Correlated_Memory_Atrophy           0.098621   0.4464  


100%|██████████| 5000/5000 [00:29<00:00, 171.85it/s]


5000
                                    Point Estimate  2.5th Percentile  \
Absolute_Correlated_Memory_Network       -0.002289         -0.098381   

                                    97.5th Percentile  P-value  
Absolute_Correlated_Memory_Network           0.083479   0.4962  


100%|██████████| 5000/5000 [00:28<00:00, 177.25it/s]

5000
             Point Estimate  2.5th Percentile  97.5th Percentile  P-value
Whole_Brain       -0.002864         -0.128024           0.112406   0.4812


Run One Thing!

In [31]:
perform_mediated_moderation_analysis(data_df, 
                                     exposure='Subiculum_Connectivity', 
                                     mediator='Baseline', 
                                     moderator='Age', 
                                     dependent_variable='outcome')

100%|██████████| 5000/5000 [00:28<00:00, 174.42it/s]

5000


,Point Estimate,2.5th Percentile,97.5th Percentile,P-value
Baseline,-0.00777,-0.152222,0.107049,0.504


**Partial Mediated Moderation**

Assess the effect of the exposure or the moderator indepdendently through the mediator upon the outcome. 

Based on Edwards and Lambert 2007, Methods for integrating moderation and mediation: A general analytical framework using moderated path analysis.

Author: Calvin Howard

NOTE: This is a proper mediated moderation analysis

Model Explanation:
A mediated moderation analysis explores how the effect of an independent variable (exposure) on a dependent variable (outcome) is moderated by a third variable, and this moderation effect is mediated by a fourth variable. Essentially, it helps to understand how the interaction between an independent variable and a moderator influences a dependent variable through a mediator. In short: is the moderation (interaction effect) attributable to something else (mediator).

Model Form:
```
Independent Variable (IV) ---(a)--> Mediator (M) ---(b)--> Dependent Variable (DV)
                      |                            ^
                      |                            |
                      |                            |
                      |---(c)----> Moderator (Mod) |
                      |                            |
                      |-----------------(d)--------|
```
In this diagram:

IV represents the independent variable.
M represents the mediator in the model.
Mod represents the moderator in the model.
DV represents the dependent variable.
The path labeled (a) represents the effect of the IV on M.
The path labeled (b) represents the effect of M on the DV, controlling for the IV and Mod.
The path labeled (c) represents the effect of the IV on Mod.
The path labeled (d) represents the interaction between IV and Mod affecting the DV, controlling for M.

Technical Notes

This is not formally implemented in Python as well as in R, so this is custom code. Use with care.

Technical Explanation
The perform_mediated_moderation_analysis function is designed to perform a mediated moderation analysis to estimate the joint indirect effects of an exposure variable through a mediator on a dependent variable, considering the moderating effect of another variable. This is achieved through the use of bootstrapping.

Bootstrapping is a statistical technique where resampling of the dataset is done with replacement to estimate the sampling distribution of a statistic. In the case of mediated moderation analysis, it's used to estimate the joint indirect effects.

In the function, for each bootstrap sample, the dataset is resampled with replacement. Then, a simple linear regression model is fitted to estimate the path from the moderator variable to the mediator, denoted as 'a' path. The interaction between exposure and mediator is estimated using a multiple linear regression model with the dependent variable as the response, and the exposure, mediator, moderator, interaction of exposure and mediator, and interaction of exposure and moderator as predictors. The product of the 'a' path and the interaction effect represent the indirect effect for this bootstrap sample.

This process is repeated for a specified number of bootstrap samples (default is 5000), and the mean indirect effect is calculated from these bootstrap samples.

Finally, the function prints the mean indirect effect and its 95% confidence interval, which is estimated using the 2.5th and 97.5th percentiles of the bootstrap indirect effects.

This approach provides an approximate estimate of the joint indirect effect through a mediator and allows for the calculation of confidence intervals around this effect.

In [ ]:
def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols
def perform_partial_mediated_moderation_analysis(dataframe, exposure, mediator, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a partial mediated moderation analysis by estimating the joint indirect effects of an exposure variable
    through a mediator on a dependent variable using bootstrapping, considering the moderating effect of another variable.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable.
    - mediator: str, column name of the mediator variable.
    - moderator: str, column name of the moderator variable.
    - dependent_variable: str, column name of the dependent variable.
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for the indirect effect.

    Example Usage:
    result_df = perform_mediated_moderation_analysis(data_df, exposure='Age',
                                                     mediator='Brain_Lobe',
                                                     moderator='Stimulation',
                                                     dependent_variable='Outcome')
    """

    ab_paths = []

    # Loop over each bootstrap sample
    for i in tqdm(range(bootstrap_samples)):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)

        # Fit the models and calculate the indirect effect for this bootstrap sample
        model_M = ols(f"{mediator} ~ {moderator}", data=sample).fit()
        model_Y = ols(f"{dependent_variable} ~ {exposure} + {mediator} + {moderator} + {exposure}:{mediator} + {exposure}:{moderator}", data=sample).fit()

        indirect_effect = model_M.params[moderator] * model_Y.params[f'{exposure}:{mediator}']

        # Append the indirect effect to the list
        ab_paths.append(indirect_effect)
    print(len(ab_paths))
    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, mediators=mediator)

    return result_df


In [ ]:
data_df.columns

In [ ]:
perform_partial_mediated_moderation_analysis(dataframe=data_df, 
                                             exposure='Subiculum_Connectivity', 
                                             mediator='Abs_Stim_Composite_Atophy_SpCorrel', 
                                             moderator='Age', 
                                             dependent_variable='outcome', 
                                             bootstrap_samples=10000)

# Conditional Process Analysis
Based on Preacher and Hayes 2008, Asymptotic Asymptotic and resampling strategies for assessing and comparing indirect effects in multiple mediator models

__________  
Multiple Mediators and Moderation Analysis
Also known as a conditional process analysis
the two code segments below will vary the moderator, 
1) allowing the moderator to interact with the exposure upon mediators,
or 
2) allowing the moderator to interact with the mediators upon outcome. 

**Multiple Mediator Analysis with First Stager Moderator**:

This code estimates the joint indirect effects of an exposure variable through multiple mediators on a dependent variable while considering the moderation effect of a specified moderator variable.

In this code, similar to the previous code, the function loops over each bootstrap sample and computes the indirect effects for each mediator, considering the moderation effect of the specified moderator variable in the first stage. It calculates the total indirect effect as the sum of the individual indirect effects for each mediator. The mean indirect effect, 95% confidence interval, and p-value are then calculated based on the bootstrap samples.

The inclusion of the moderator variable and its interaction with the exposure variable in the first stage allows for examining how the mediation effects may vary based on different levels of the moderator. By considering the moderation effect at the first stage, this analysis investigates the conditional indirect effects of the exposure through the mediators.


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def perform_multiple_mediation_first_stage_moderator_analysis(dataframe, exposure, mediators, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a multiple mediation analysis by estimating the joint indirect effects of an exposure variable
    through multiple mediators on a dependent variable using bootstrapping.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable (e.g., 'Age').
    - mediators: list, column names of the mediator variables (e.g., ['Brain_Lobe1', 'Brain_Lobe2']).
    - moderator: str, column name of the moderator variable.
    - dependent_variable: str, column name of the dependent variable (e.g., 'outcome').
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - None. Prints the mean indirect effect and 95% confidence interval.

    Example Usage:
    perform_multiple_mediation_analysis(data_df, exposure='Age', mediators=['Brain_Lobe1', 'Brain_Lobe2'], dependent_variable='outcome')
    """

    ab_paths, total_indirect_effects = [], []
    
    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)
        # Paths from exposure to mediators and from mediators to DV
        ab_paths_sample = [ols(f"{mediator} ~ {exposure} + {moderator} + {exposure}:{moderator}", data=sample).fit().params[exposure] * 
                    ols(f"{dependent_variable} ~ {exposure} + {moderator} + {exposure}:{moderator} + {mediator}", data=sample).fit().params[mediator]
                    for mediator in mediators]
        
        # Sum the individual indirect effects for this bootstrap sample
        total_indirect_effect = sum(ab_paths_sample)
        ab_paths.append(ab_paths_sample)
        total_indirect_effects.append(total_indirect_effect)
    
    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, total_indirect_effects, mediators)

    return result_df


In [ ]:
print(data_df.columns)

In [ ]:
perform_multiple_mediation_first_stage_moderator_analysis(data_df, 
                                                    exposure='Age', 
                                                    moderator='Subiculum_Connectivity', 
                                                    mediators= ['Hippocampus', 'Cerebellum', 'Frontal', 'Parietal',
       'Temporal', 'Occipital'], 
                                                    dependent_variable='outcome',
                                                    bootstrap_samples=10000)

--------------------
Moderated Mulitple Mediation Analysis, but with all mediators combined instead of run individually
- This is not supported natively in Python libraries, so I have developed it myself. 

Experimental, please use with caution. 

_____
**Multiple Mediator Analysis with Second Stage Moderator**


In this code, the function loops over each bootstrap sample and computes the indirect effects for each mediator, taking into account the moderation effect of the specified moderator variable. It calculates the total indirect effect as the sum of the individual indirect effects for each mediator. The mean indirect effect, 95% confidence interval, and p-value are then calculated based on the bootstrap samples.

The addition of the moderator variable in the second-stage analysis allows for assessing how the mediation effects may vary across different levels of the moderator. By including the moderator variable and its interaction with the mediators, the analysis investigates whether the indirect effects of the exposure through the mediators differ depending on the levels of the moderator.

Overall, this code provides a way to examine multiple mediation effects while considering a second-stage moderation analysis to explore potential moderation effects.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def perform_multiple_mediation_second_stage_moderator_analysis(dataframe, exposure, mediators, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a multiple mediation analysis by estimating the joint indirect effects of an exposure variable
    through multiple mediators on a dependent variable using bootstrapping.

    Parameters:
    - dataframe: DataFrame containing the data.
    - exposure: str, column name of the exposure variable (e.g., 'Age').
    - mediators: list, column names of the mediator variables (e.g., ['Brain_Lobe1', 'Brain_Lobe2']).
    - moderator: str, column name of the moderator variable.
    - dependent_variable: str, column name of the dependent variable (e.g., 'outcome').
    - bootstrap_samples: int, optional, number of bootstrap samples to be used (default is 5000).

    Returns:
    - None. Prints the mean indirect effect and 95% confidence interval.

    Example Usage:
    perform_multiple_mediation_analysis(data_df, exposure='Age', mediators=['Brain_Lobe1', 'Brain_Lobe2'], dependent_variable='outcome')
    """

    ab_paths, total_indirect_effects = [], []
    
    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)
        # Paths from exposure to mediators and from mediators to DV
        ab_paths_sample = [ols(f"{mediator} ~ {exposure}", data=sample).fit().params[exposure] * 
                    ols(f"{dependent_variable} ~ {exposure} + {moderator} + {mediator}:{moderator} + {mediator}", data=sample).fit().params[mediator]
                    for mediator in mediators]
        
        # Sum the individual indirect effects for this bootstrap sample
        total_indirect_effect = sum(ab_paths_sample)
        ab_paths.append(ab_paths_sample)
        total_indirect_effects.append(total_indirect_effect)
    
    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, total_indirect_effects, mediators)


    return result_df


In [ ]:
perform_multiple_mediation_second_stage_moderator_analysis(data_df, 
                                               exposure = 'Age', 
                                               moderator = 'Subiculum_Connectivity', 
                                               mediators= ['Hippocampus', 'Cerebellum', 'Frontal', 'Parietal',
       'Temporal', 'Occipital'], 
                                                    dependent_variable='outcome',
                                                    bootstrap_samples=10000)

In [ ]:
data_df.columns

In [ ]:
results_df = perform_mediated_moderation_analysis(dataframe = data_df,
                                                  exposure = 'Subiculum_Connectivity', 
                                                  mediator = 'Subiculum_Damage_Score', 
                                                  moderator = 'Age', 
                                                  dependent_variable ='outcome', 
                                                  bootstrap_samples=5000)
results_df

# Complex Models

**Mediated Moderator Analysis AKA Second Stage Moderation Mediation**

The indirect effect of the moderator on the outcome through the mediator is conditional on the exposure and its interactions with the mediator and the moderator.
_____

Let's break this down:

Similar to my previous examples, we will use the same variables. This model suggests that:

Age (the initial moderator) interacts with DBS (the exposure) to influence the outcome.
However, the moderation effect of age might not be the "true" or direct moderation. Instead, it might be that something underlying age (another variable) is what's truly driving this moderation.
In terms of a pathway:

DBS influences the outcome.
The effect of DBS on the outcome varies based on age.
However, the moderating effect of age is itself influenced (or mediated) by another variable. This other variable could be a more direct measure or something that captures the biological, psychological, or physiological changes that come with age.
Here's a step-by-step breakdown of the model:

Mediator Model (for the "true" underlying mediator of age's moderation):
- Mediator is predicted by age.

Outcome Model:
- Outcome is predicted by DBS, the mediator, age, and interactions between DBS and both age and the mediator.
This model seeks to understand if the interaction between age and DBS is not just because of chronological age itself but due to some underlying factor associated with age.

Is it just a mediated moderation?
It's close but slightly different. In a typical mediated moderation:

The exposure affects the mediator.
The mediator, the exposure, and their interaction influence the outcome.In the Second Stage Moderation Mediation:

Age (the moderator) affects the mediator.
The mediator, the exposure (DBS), age, and their interactions influence the outcome.

In [35]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.mediation import Mediation

def mediated_moderator_analysis(dataframe, exposure, mediator, moderator, dependent_variable, bootstrap_samples=5000):
    """
    Performs a mediated moderator analysis.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for the indirect effect.

    Example Usage:
    result_df = mediated_moderator_analysis(data_df, exposure='DBS',
                                            mediator='UnderlyingVariable',
                                            moderator='Age',
                                            dependent_variable='Outcome')
    """

    ab_paths = []

    # Loop over each bootstrap sample
    for i in range(bootstrap_samples):
        # Resample the data with replacement
        sample = dataframe.sample(frac=1, replace=True)

        # Fit the mediator model: Mediator ~ Moderator
        model_M = ols(f"{mediator} ~ {moderator}", data=sample).fit()

        # Fit the outcome model: Outcome ~ Exposure + Mediator + Moderator + Exposure:Mediator + Exposure:Moderator
        model_Y = ols(f"{dependent_variable} ~ {exposure} + {mediator} + {moderator} + {exposure}:{mediator} + {exposure}:{moderator}", data=sample).fit()

        # Calculate the indirect effect for this bootstrap sample
        indirect_effect = model_M.params[moderator] * (model_Y.params[exposure] + model_Y.params[f'{exposure}:{mediator}'])

        # Append the indirect effect to the list
        ab_paths.append(indirect_effect)

    # Calculate confidence intervals and p-values
    result_df = calculate_confidence_intervals(ab_paths, mediators=mediator)

    return result_df


In [36]:
data_df.columns

Index(['subject_id', 'Age', 'Baseline', 'Subiculum_Connectivity',
       'Abs_Stim_Composite_Atophy_SpCorrel',
       'Raw_Stim_Composite_Atrophy_SpCorrel', 'Frontal_Atrophy',
       'Insula_Atrophy', 'Temporal_Atrophy', 'Cerebellar_Atrophy',
       'Subiculum_Atrophy', 'Occipital_Atrophy', 'Parietal_Atrophy',
       'Total_Atrophy_Voxels', 'Visual_Connectivity',
       'Somatomotor_Connectivity', 'Dorsal_Attention_Connectivity',
       'Ventral_Attention_Connectivity', 'Limbic_Connectivity',
       'Frontoparietal_Connectivity', 'Default_Connectivity',
       'Default_Atrophy', 'Visual_Atrophy', 'Limbic_Atrophy',
       'Somatomotor_Atrophy', 'Dorsal_Attention_Atrophy',
       'Ventral_Attention_Atrophy', 'Frontoparietal_Atrophy',
       'Null_Memory_Atrophy', 'Anticorrelated_Memory_Atrophy',
       'Correlated_Memory_Atrophy', 'Absolute_Correlated_Memory_Network',
       'Whole_Brain', 'outcome'],
      dtype='object')

In [40]:
mediated_moderator_analysis(dataframe=data_df, 
                            exposure='Subiculum_Connectivity', 
                            mediator='Whole_Brain', 
                            moderator='Age', 
                            dependent_variable='outcome', 
                            bootstrap_samples=5000)

,Point Estimate,2.5th Percentile,97.5th Percentile,P-value
Whole_Brain,0.023983,-0.122532,0.237675,0.417
